In [74]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os
from PIL import Image

In [75]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
def delete_zone_identifier(filenames, directory):
    for filename in filenames:
        try:
            name = filename.split(":")[1]
        except:
            name = ""
        if name == "Zone.Identifier":
            os.remove(directory + "/" + filename)

In [77]:
filenames_train = os.listdir("train/train")
delete_zone_identifier(filenames_train, "train/train")
filenames_test = os.listdir("test1/test1")
delete_zone_identifier(filenames_test, "test1/test1")

In [78]:
def preprocess_image(file_path, target_size):
    image = Image.open(file_path)
    image = image.convert('RGB')
    image = image.resize(target_size)
    image = np.array(image, dtype="float32") / 255.0
    return image

In [79]:
def parse_images(filenames, directory, limit_dogs, limit_cats):
    for filename in filenames:
        category = filename.split('.')[0]
        if category == "dog":
            limit_dogs -= 1
            if limit_dogs < 0:
                os.remove(directory + "/" + filename)
        else:
            limit_cats -= 1
            if limit_cats < 0:
                os.remove(directory + "/" + filename)

In [80]:
filenames_train = os.listdir("train/train")
parse_images(filenames_train, "train/train", 500, 500)
filenames_test = os.listdir("test1/test1")
parse_images(filenames_test, "test1/test1", 100, 100)

In [81]:
def load_dataset(filenames):
    images = []
    labels = []
    for filename in filenames:
        image = preprocess_image("train/train/" + filename, (64, 64))
        images.append(image)
        category = filename.split('.')[0]
        if category == "dog":
            labels.append(1)
        else:
            labels.append(0)
    return np.array(images), np.array(labels)

In [82]:
filenames = os.listdir("train/train")
train_set_x_orig, train_set_y = load_dataset(filenames_train)

In [83]:
train_set_x = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
train_set_y = train_set_y.reshape(1, train_set_y.shape[0])

In [84]:
train_set_x.shape, train_set_y.shape

((12288, 1000), (1, 1000))

In [86]:
LAYER_DIMS = [7, 5, 3]

In [88]:
def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters

In [89]:
parameters = initialize_parameters(LAYER_DIMS)
print(parameters)

{'W1': array([[-0.00012665, -0.0111731 ,  0.00234416,  0.01659802,  0.00742044,
        -0.00191836, -0.00887629],
       [-0.00747158,  0.01692455,  0.00050808, -0.00636996,  0.00190915,
         0.02100255,  0.00120159],
       [ 0.00617203,  0.0030017 , -0.0035225 , -0.01142518, -0.00349343,
        -0.00208894,  0.00586623],
       [ 0.00838983,  0.00931102,  0.00285587,  0.00885141, -0.00754398,
         0.01252868,  0.0051293 ],
       [-0.00298093,  0.00488518, -0.00075572,  0.01131629,  0.01519817,
         0.02185575, -0.01396496]]), 'b1': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]]), 'W2': array([[-0.01444114, -0.00504466,  0.00160037,  0.00876169,  0.00315635],
       [-0.02022201, -0.00306204,  0.00827975,  0.00230095,  0.00762011],
       [-0.00222328, -0.00200758,  0.00186561,  0.00410052,  0.001983  ]]), 'b2': array([[0.],
       [0.],
       [0.]])}


In [106]:
def RELU(Z):
    cache = Z
    A = np.maximum(Z, 0)
    return A, cache

In [107]:
def sigmoid(Z):
    cache = Z
    A = 1 / (1 + np.exp(-A))
    return A, cache

In [100]:
def linear_forward(A, W, b):
    Z = np.matmul(A, W) + b
    cache = (A, W, b)
    return Z, cache

In [102]:
def linear_activation_forward(prev_A, W, b, activ_funct):
    if activ_funct == "RELU":
        Z, linear_cache = linear_forward(prev_A, W, b)
        A, activation_cache = RELU(Z)

    elif activ_funct == "sigmoid":
        Z, linear_cache = linear_forward(prev_A, W, b)
        A, activation_cache = sigmoid(Z)
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [105]:
def forward_propagation(X, parameters):
    caches = []
    L = len(parameters) //2
    A_prev = X

    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b'+str(l)], "RELU")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)], "sigmoid")
    caches.append(cache)

    return AL, caches

In [104]:
def cost_function(AL, Y):
    m = Y.shape[1]
    element1 = np.multiply(Y, np.log(AL))
    element2 = np.multiply((1 - Y), np.log(1 - AL))
    cost = (-1 / m) * np.sum(element1, element2)

    cost = np.squeeze(cost)

    return cost

In [108]:
def RELU_backward(dA, activ_cache):
    Z = activ_cache
    dZ = np.array(dA, copy=True)

    if  Z <= 0:
        dZ =0
    return dZ

In [109]:
def sigmoid_backward(dA, activ_cache):
    Z = activ_cache

    x = 1 / (1 + np.exp(-Z))
    dZ = dA * x * (1-x)

    return dZ

In [111]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = (1 / m) * np.matmul(dZ, A_prev)
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.matmul(W, dZ)

    return dA_prev, dW, db

In [112]:
def linear_activation_backward(dA, cache, activ_funct):
    linear_cache, activ_cache = cache
    if activ_funct == "relu":
        dZ = RELU_backward(dA, activ_cache)
    elif activ_funct == "sigmoid":
        dZ = sigmoid_backward(dA, activ_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db